<a href="https://colab.research.google.com/github/GabrielMtzSoltero/wsEMBS/blob/main/workshopEMBS_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [7]:
with open('sample_data/kmer_str_multilabel2.pkl', 'rb') as f:
     df=pickle.load(f)
     df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   kmer_id           12000 non-null  int64 
 1   kmer_str          12000 non-null  object
 2   kmer_gocom_id_fk  12000 non-null  int64 
 3   0000287           12000 non-null  int64 
 4   0000166           12000 non-null  int64 
 5   0000027           12000 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 562.6+ KB


In [8]:
df.head()

,kmer_id,kmer_str,kmer_gocom_id_fk,0000287,0000166,0000027
0,1070986,N O G 0 A A Y G G N A A R,3915,1,0,0
1,1071016,R O G 0 T A Y J T 0 T G Y,3915,1,0,0
2,1071037,Y O G 0 A A R A C N T G G,3915,1,0,0
3,1071130,Y O G 0 G C N A C N G C N,3915,1,0,0
4,1071142,N O G 0 A U H A T G G G N,3915,1,0,0


In [9]:
strings=df["kmer_str"]

In [10]:
strings=[kmer.split(" ") for kmer in strings]
df2 = pd.DataFrame(strings, columns =['kmer1', 'kmer2','kmer3', 'kmer4', 'kmer5', 'kmer6','kmer7','kmer8', 'kmer9','kmer10','kmer11','kmer12','kmer13'])
df2.tail()

,kmer1,kmer2,kmer3,kmer4,kmer5,kmer6,kmer7,kmer8,kmer9,kmer10,kmer11,kmer12,kmer13
11995,R,J,T,0,A,A,Y,A,U,H,G,A,R
11996,N,J,T,0,G,C,N,G,G,N,C,A,Y
11997,N,J,T,0,G,G,N,J,T,0,T,T,Y
11998,N,J,T,0,T,T,Y,G,G,N,A,G,Y
11999,R,J,T,0,G,G,N,T,G,G,G,A,Y


In [11]:
df=pd.concat([df, df2],axis=1)
df.tail()

,kmer_id,kmer_str,kmer_gocom_id_fk,0000287,0000166,0000027,kmer1,kmer2,kmer3,kmer4,kmer5,kmer6,kmer7,kmer8,kmer9,kmer10,kmer11,kmer12,kmer13
11995,1128502,R J T 0 A A Y A U H G A R,3985,0,1,1,R,J,T,0,A,A,Y,A,U,H,G,A,R
11996,1128528,N J T 0 G C N G G N C A Y,3985,0,1,1,N,J,T,0,G,C,N,G,G,N,C,A,Y
11997,1128570,N J T 0 G G N J T 0 T T Y,3985,0,1,1,N,J,T,0,G,G,N,J,T,0,T,T,Y
11998,1128575,N J T 0 T T Y G G N A G Y,3985,0,1,1,N,J,T,0,T,T,Y,G,G,N,A,G,Y
11999,1128609,R J T 0 G G N T G G G A Y,3985,0,1,1,R,J,T,0,G,G,N,T,G,G,G,A,Y


In [12]:
vocab=np.load("sample_data/vocab.npy")
print(vocab)
vocab=[str(element) for element in vocab]
df=df.replace(vocab,[i for i in range(1,len(vocab)+1)])
df.head()

['A' 'O' 'R' 'N' 'C' 'Y' 'T' '0' 'G' 'J' 'U' 'X' 'H' 'B' 'Z']


,kmer_id,kmer_str,kmer_gocom_id_fk,0000287,0000166,0000027,kmer1,kmer2,kmer3,kmer4,kmer5,kmer6,kmer7,kmer8,kmer9,kmer10,kmer11,kmer12,kmer13
0,1070986,N O G 0 A A Y G G N A A R,3915,1,0,0,4,2,9,8,1,1,6,9,9,4,1,1,3
1,1071016,R O G 0 T A Y J T 0 T G Y,3915,1,0,0,3,2,9,8,7,1,6,10,7,8,7,9,6
2,1071037,Y O G 0 A A R A C N T G G,3915,1,0,0,6,2,9,8,1,1,3,1,5,4,7,9,9
3,1071130,Y O G 0 G C N A C N G C N,3915,1,0,0,6,2,9,8,9,5,4,1,5,4,9,5,4
4,1071142,N O G 0 A U H A T G G G N,3915,1,0,0,4,2,9,8,1,11,13,1,7,9,9,9,4


In [13]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
X=df.drop(["kmer_id","kmer_str","kmer_gocom_id_fk","0000287","0000166","0000027"], axis = 1)
y = np.asarray(df[df.columns[3:6]])
#creamos el escalador
scaler = StandardScaler()
scaledXDF=X.copy()
#escalar los valores de entradas
scaledXDF[X.columns]=scaler.fit_transform(X)
#SPlitting into train and test
X_train, X_test, y_train, y_test = train_test_split(scaledXDF, y, test_size = 0.33, random_state=1)
model = MultiOutputClassifier(LogisticRegression()).fit(X_train, y_train)
y_pred = model.predict(X_test)
print("score train ",model.score(X_train, y_train))
print("score_test ",model.score(X_test, y_test))

score train  0.9401741293532339
score_test  0.9356060606060606


In [14]:
from sklearn.metrics import hamming_loss
print('Hamming Loss: ', hamming_loss(y_test, y_pred))

Hamming Loss:  0.021464646464646464
